# Чтение данных

In [1]:
import pandas as pd
import re

from tqdm import tqdm
tqdm.pandas()

In [2]:
df_list = []

for i in range(12):
    df_list.append(
        pd.read_csv(f"../collecting/columns_headers/headers_{i}.csv", sep="⇧", engine="python", quotechar='"', on_bad_lines="warn")
    )
df = pd.concat(df_list, axis=0)
df

Skipping line 49505: '⇧' expected after '"'
Skipping line 49506: '⇧' expected after '"'
Skipping line 49507: '⇧' expected after '"'
Skipping line 345854: field larger than field limit (131072)


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0,1,Высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,Год
3,dataset/Dataset/data/4022432/table_0,1,Unnamed: 1
4,dataset/Dataset/data/4022432/table_0,2,Русское название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,Дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,Сезон
411441,dataset/Dataset/data/6206410/table_0,1,Короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


# Research
---
Проблемные столбцы (скорее всего не будем использовать):
- Столбцы по типу: "ΔHкип, кДж·моль−1"
- Столбцы использующиеся для группировки каких-либо значений по времени (месяцу, году и т.п.)
- Столбцы без имени
- Столбцы, название которых не имеет смысла (числа и т.п.)
- Аббревиатуры и сокращения

### Столбцы без названия = 164500
их можно отправить в тестовую выборку, чтобы не терять информацию. (нужно еще посмотреть, что внутри них обычно лежит)

### Без имени
можно в тест

## .mv-parser... = 15661
обрезать только осмысленный текст

---

In [3]:
df["column_name"].value_counts()

column_name
Год                    220601
Место                   87167
Роль                    78957
Команда                 67698
Название                64939
                        ...  
Со-производитель(и)         1
Области применения          1
51 220                      1
1999—2001                   1
1724                        1
Name: count, Length: 125673, dtype: int64

## Столбцы без названия

In [4]:
unnamed_columns = df["column_name"].apply(lambda x: str(x).startswith("Unnamed"))
df[unnamed_columns]

,table_id,column_id,column_name
3,dataset/Dataset/data/4022432/table_0,1,Unnamed: 1
215,dataset/Dataset/data/8128699/table_0,1,Unnamed: 1
225,dataset/Dataset/data/7786517/table_0,1,Unnamed: 1
267,dataset/Dataset/data/3595819/table_2,0,Unnamed: 0
324,dataset/Dataset/data/1802083/table_7,0,Unnamed: 0
...,...,...,...
411160,dataset/Dataset/data/8352540/table_7,1,Unnamed: 1
411268,dataset/Dataset/data/2584082/table_1,2,Unnamed: 2
411275,dataset/Dataset/data/2584082/table_1,9,Unnamed: 9
411289,dataset/Dataset/data/980012/table_0,0,Unnamed: 0


## Столбцы содержащие в названии: ".mw-parser..."

In [5]:
df["column_name"].isna().sum()

36

In [6]:
df = df[~df["column_name"].isna()]
mw_parser = df["column_name"].str.contains(".mw-parser")
mw_parser.sum()

15771

### В назнваниях столбца содержится человекопонятное название, нужно отчистить мусорную информацию

In [7]:
mw_parser_humnareadable_names = df[mw_parser]["column_name"].apply(lambda x: str(x).split("}")[-1])
mw_parser_humnareadable_names.unique()

array(['М', 'МУ', 'Цвета участниц', 'Дата пожалования', '#', 'Ref.', 'ХП',
       '№', 'Жертвы', '7,5+7,5 км', 'Коэфф.', 'В', 'И', 'N', 'Коэф.',
       'Д / В', 'Год', 'П', '№ п/п', ' В', '09/10', 'Bib', 'Прим.',
       '02/03', 'Пер.', 'Сезоны', 'Категория', 'Кубок', 'С', 'И.', 'БК',
       'ТТХ', 'Уров.', 'HP', 'Поз', 'Тренер', 'КЧ', 'КП', 'Поз.', 'СУ',
       'Н', 'Доступ-ность', 'Гр.', 'ПЧ', '08/09', '11/12',
       'Country Songs[A]', 'Д / В / Н)', 'Д / В / Н).1', '±', 'З',
       'Должность / Титул', 'Ном.', 'ПП', 'д/г', 'Этап', '15+15 км',
       ' От', 'СУ, км', ' И', 'К-во', 'GP', 'Очки', 'И, С', 'П.',
       'Численностьверующихмлн. человек', 'Период', 'ГП[67]',
       'Дата рождения', 'Нац', '+/-', '06/07', 'ед. изм.',
       'Оценка за трудность',
       'Доля учеников по обучению на русском и белорусском языках,\xa0%[42][39][43]:',
       '04/05', 'Ист.', 'Кр', 'ЧС', 'КНСО', 'Голосовало', 'Рез.', 'SP',
       'SP.1', 'Норвежское название', 'Оч', 'Л/Круги', 'Оценка D', 'RPG

## Аббревиатуры / сокращения / меры (кг, кдж.)

In [8]:
# TODO: Мы их не рассматриваем, потому что в столбце будет то же что угодно

In [9]:
# no_word_in_header = df_cleansed["column_name"].str.len() < 2
# df_cleansed[no_word_in_header]["column_name"]

# Очистка данных

---
1. Очистка мусора из столбцов, содержащих ".mv-parser.."
2. Выделение столбцов без заголовка
3. Замена специальных символов на слова (№, $, %, и т.п.)
4. Очистка символов не из русского алфавита
5. Удаление лишних пробелов с начала и конца заголовка
---

In [10]:
df_cleansed = df.copy()

In [11]:
df_cleansed = df[~df["column_name"].isna()]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0,1,Высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,Год
3,dataset/Dataset/data/4022432/table_0,1,Unnamed: 1
4,dataset/Dataset/data/4022432/table_0,2,Русское название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,Дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,Сезон
411441,dataset/Dataset/data/6206410/table_0,1,Короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


## 1. Очистка мусора из столбцов

In [12]:
# def clean_mv_parser(col_name: str) -> str:
#     if col_name.strip().startswith(".mw-parser"):
#         return col_name.split("}")[-1]
#     return col_name

# df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: clean_mv_parser(str(x)))

In [13]:
df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(
    lambda x: re.sub(r"\.mw-parser-output \.ts.+{.+}", "", x)
)
df_cleansed["column_name"].str.contains("mw-parser").sum()

100%|████████████████████████████| 4965670/4965670 [00:03<00:00, 1506397.89it/s]


0

In [14]:
df_cleansed[mw_parser]

,table_id,column_id,column_name
52,dataset/Dataset/data/4659559/table_0,0,М
605,dataset/Dataset/data/5220971/table_15,1,МУ
1437,dataset/Dataset/data/5346766/table_1,0,М
1777,dataset/Dataset/data/4747431/table_1,0,М
2040,dataset/Dataset/data/7955239/table_5,0,М
...,...,...,...
410876,dataset/Dataset/data/5375371/table_0,1,И
410941,dataset/Dataset/data/5372811/table_0,1,И
410988,dataset/Dataset/data/2554658/table_0,0,Поз.
411366,dataset/Dataset/data/5451199/table_0,0,М


## 2. Выделение столбцов без заголовка

In [15]:
unnamed_columns = df_cleansed["column_name"].str.startswith("Unnamed")
df_cleansed = df_cleansed[~unnamed_columns]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0,1,Высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,Год
4,dataset/Dataset/data/4022432/table_0,2,Русское название
5,dataset/Dataset/data/4022432/table_0,3,Оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,Дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,Сезон
411441,dataset/Dataset/data/6206410/table_0,1,Короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


## 3. Замена специальных символов на слова

In [16]:
def remove_alphabetic_symbols(col_name: str) -> str:
    """TODO: just replace this symbols with space"""
    restricted_symbols = [
        "(", 
        ")", 
        "+", 
        "-", 
        "[", 
        "]", 
        "/", 
        ".", 
        ",", 
        "@", 
        "{", 
        "}", 
        "«", 
        "»", 
        "<", 
        ">", 
        ":", 
        ";", 
        "'",
        "*",
        "?",
        "!",
        "-",
        "_",
    ]
    return "".join(c for c in col_name if not c.isalpha() and not c.isdigit() and not c.isspace() and c not in restricted_symbols)

non_alpha = df_cleansed["column_name"].progress_apply(lambda x: remove_alphabetic_symbols(str(x)))
non_alpha.unique()

100%|████████████████████████████| 4801230/4801230 [00:04<00:00, 1017329.16it/s]


array(['', '№', '±', '#', '—', '%', '$', '\\№', '☉', '•', '−', '=', '""',
       '–', '⊙', '\\', '°', '·−', '́', '€', '\xad', '’%', '–%', '∅', '≈',
       '&', '|', '%%', '%%%%———————————', '\xad\xad', '∧', '••', '→↓',
       '₽', '̱', '½', '≥', '\x7f"``"\x7f', '·', '÷%', '…', '͡', '≡', '‰',
       '↓→', '——', '&&', '’', '=""', '̨', '’’', '→', '×', '~', '↓', '⋅',
       '№№', '°·', '£', '±%', '$$', '†', '—°', '↕', '̃', '•—', '%~', '––',
       'ं', 'ः', '%—', '‡', '×=', '====', '№№——№—№——№——№№—№—№№—', '्',
       '½½', '∑', '¶', '¥', '′', '%%%%%%', '||', '××', '^', '"', '−·',
       '⊕', '￼', '♂', '♀', '−−', '´', '˘', '———', '∆%', '（）', '%%%%%%%%',
       '¼', '̱̱', '%±', 'ꤦ', 'ꤧ', 'ꤨ', 'ꤩ', 'ꤪ', '★', '№\\',
       '\xad\xad\xad\xad\xad', '\xad\xad\xad\xad', '∙−', '∙∙−', '≤°',
       '≥°', '▼', 'ಂ', 'ಃ', '∞∞', '""""', '¬', '∞', '♁', '——————',
       '———————', '\xad\xad\xad', '……………………', '=%', '↑', '▼▲%',
       '№№—№—————————', '—%', '=−', '\u200e', '%№', '′′', '’—', '‘…’',
       '⠀'

In [17]:
# # - тоже номер или не всегда
# Много знаков равенства, скорее всего в столбце число лежит, можно тоже указать
rules = {
    "№": " номер ",
    "%": " процент ",
    "∆": " число ",
    "∑": " сумма ",
    "Σ": " сумма ",
    "$": " деньги ",
    "₣": " деньги ",
    "₽": " деньги ",
    "€": " деньги ",
    "¥": " деньги ",
    "₩": " деньги ",
    "£": " деньги ",
}

for rule, word in rules.items():
    df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).replace(rule, word))

df_cleansed

100%|████████████████████████████| 4801230/4801230 [00:01<00:00, 2737262.83it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0,1,Высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,Год
4,dataset/Dataset/data/4022432/table_0,2,Русское название
5,dataset/Dataset/data/4022432/table_0,3,Оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,Дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,Сезон
411441,dataset/Dataset/data/6206410/table_0,1,Короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


## 3.1. To lower

In [18]:
df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).lower())
df_cleansed

100%|████████████████████████████| 4801230/4801230 [00:02<00:00, 2319276.98it/s]
/tmp/ipykernel_18476/2647402885.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).lower())


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт (2019)
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
4,dataset/Dataset/data/4022432/table_0,2,русское название
5,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,сезон
411441,dataset/Dataset/data/6206410/table_0,1,короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,произвольная программа


## 3.2 замена ё на е

In [19]:
df_cleansed["column_name"] = df_cleansed["column_name"].str.replace("ё", "е")

/tmp/ipykernel_18476/1329648267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleansed["column_name"] = df_cleansed["column_name"].str.replace("ё", "е")


## 4. Удаление символов не из русского алфавита

In [20]:
# for i in df_cleansed.sort_values(by=["column_name"], ascending=False)["column_name"]:
#     print(i)

In [21]:
def remove_non_alphabetic_symbols(col_name: str) -> str:
    # return "".join(c for c in col_name if c.isalpha() or c == " ")
    # return "".join(c for c in col_name if "А" <= c <= "я" or c == " ")
    for c in col_name:
        if not ("а" <= c <= "я" or c == " "):
            col_name = col_name.replace(c, " ")
    return col_name
    # return "".join(c for c in col_name if "а" <= c <= "я" or c == " ")

df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: remove_non_alphabetic_symbols(str(x)))
df_cleansed

100%|████████████████████████████| 4801230/4801230 [00:03<00:00, 1281405.27it/s]
/tmp/ipykernel_18476/919542367.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: remove_non_alphabetic_symbols(str(x)))


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
4,dataset/Dataset/data/4022432/table_0,2,русское название
5,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,сезон
411441,dataset/Dataset/data/6206410/table_0,1,короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [22]:
len_le_2 = df_cleansed["column_name"].str.len() == 0
df_cleansed[len_le_2]

,table_id,column_id,column_name
80632,dataset/Dataset/data/8289176/table_1,7,
378460,dataset/Dataset/data/8113812/table_1,7,


## 5. Удаление пробелов с начала и конца заголовка столбца

In [23]:
df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).strip())
df_cleansed = df_cleansed[df_cleansed["column_name"] != ""]

100%|████████████████████████████| 4801230/4801230 [00:01<00:00, 2926377.67it/s]


## 6. Удаление заголовков столбцов, используемых для группировки значений (по месяцу, году и т.п.)

In [24]:
group_by_columns = [
    "январь",
    "февраль",
    "март",
    "апрель",
    "май",
    "июнь",
    "июль",
    "август",
    "сентябрь",
    "октябрь",
    "ноябрь",
    "декабрь",
    "янв",
    "февр",
    "фев",
    "мар",
    "апр",
    "июн",
    "июл",
    "авг",
    "сент",
    "сен",
    "окт",
    "ноя",
    "нояб",
    "дек",
]

In [25]:
s = 0
for col in group_by_columns:
    s += df_cleansed[df_cleansed["column_name"] == col].shape[0]
s

137387

In [26]:
for col in group_by_columns:
    group_by_mask = df_cleansed["column_name"] == col
    df_cleansed = df_cleansed[~group_by_mask]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
4,dataset/Dataset/data/4022432/table_0,2,русское название
5,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,сезон
411441,dataset/Dataset/data/6206410/table_0,1,короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [27]:
(df_cleansed["column_name"] == "фев").sum()

0

## 7. Удаление заголовков длиной <= 2

TODO: split count max len of word; if <= 2 -> drop

In [28]:
len_le_2 = df_cleansed["column_name"].str.len() <= 2
df_cleansed[len_le_2]

,table_id,column_id,column_name
47,dataset/Dataset/data/7873710/table_0,2,пу
52,dataset/Dataset/data/4659559/table_0,0,м
54,dataset/Dataset/data/4659559/table_0,2,и
55,dataset/Dataset/data/4659559/table_0,3,в
56,dataset/Dataset/data/4659559/table_0,4,н
...,...,...,...
411413,dataset/Dataset/data/3818327/table_0,2,и
411414,dataset/Dataset/data/3818327/table_0,3,в
411415,dataset/Dataset/data/3818327/table_0,4,н
411416,dataset/Dataset/data/3818327/table_0,5,п


In [29]:
df_cleansed = df_cleansed[~len_le_2]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
4,dataset/Dataset/data/4022432/table_0,2,русское название
5,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,сезон
411441,dataset/Dataset/data/6206410/table_0,1,короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [30]:
df_cleansed = df_cleansed.reset_index().drop(["index"], axis=1)
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
3,dataset/Dataset/data/4022432/table_0,2,русское название
4,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
3381700,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
3381701,dataset/Dataset/data/6206410/table_0,0,сезон
3381702,dataset/Dataset/data/6206410/table_0,1,короткая программа
3381703,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [31]:
from random import randint

rnd = randint(0, df_cleansed.shape[0])
df_cleansed.loc[rnd:rnd+10]

,table_id,column_id,column_name
2396158,dataset/Dataset/data/4738712/table_2,1,дата
2396159,dataset/Dataset/data/4738712/table_2,2,соперник
2396160,dataset/Dataset/data/4738712/table_2,3,д г
2396161,dataset/Dataset/data/4738712/table_2,4,счет
2396162,dataset/Dataset/data/4738712/table_2,5,зрителей
2396163,dataset/Dataset/data/4738712/table_2,6,в п
2396164,dataset/Dataset/data/7799219/table_4,0,марта норвегия протокол финлянд...
2396165,dataset/Dataset/data/6786161/table_0,0,результат
2396166,dataset/Dataset/data/6786161/table_0,1,год
2396167,dataset/Dataset/data/6786161/table_0,2,турнир


# Сохранение очищенного датафрейма

In [32]:
import numpy as np

df_cleansed.to_csv(f"./cleansed/data.csv", sep=";", index=False)

# number_of_chunks = 10
# for id, chunk in enumerate(np.array_split(df_cleansed, number_of_chunks)):
#     chunk.to_csv(f"./cleansed/data_{id}.csv", sep=";", index=False)

# Лемматизация

Дальнейшая разметка строится на предположении, что первое существительное в заголовке столбца отражает ту сущность, которая в нем содержится

In [33]:
import spacy

nlp = spacy.load("ru_core_news_sm", disable=['parser', 'ner'])

def lemmatize_word(word: str) -> str | None:
    for token in nlp(word):
        if token.pos_ == "NOUN":
            return token.lemma_
    return None

In [34]:
column_name_lemma = pd.DataFrame()
column_name_lemma["column_name"] = df_cleansed["column_name"].unique()
column_name_lemma["column_name_lemma"] = column_name_lemma["column_name"].progress_apply(lambda x: lemmatize_word(x))
column_name_lemma

100%|████████████████████████████████████| 63244/63244 [01:44<00:00, 606.49it/s]


,column_name,column_name_lemma
0,чарт,чарт
1,высшаяпозиция,высшаяпозиция
2,год,год
3,русское название,название
4,оригинальное название,название
...,...,...
63239,раунды время,раунд
63240,команда альфредссона,команда
63241,объем производства железной руды млн тонн,объём
63242,время выпуска,время


In [35]:
df_cleansed_lemma = df_cleansed.merge(column_name_lemma, how="inner")
df_cleansed_lemma

,table_id,column_id,column_name,column_name_lemma
0,dataset/Dataset/data/7779522/table_0,0,чарт,чарт
1,dataset/Dataset/data/1888657/table_1,2,чарт,чарт
2,dataset/Dataset/data/6223308/table_0,0,чарт,чарт
3,dataset/Dataset/data/4293840/table_0,0,чарт,чарт
4,dataset/Dataset/data/8245389/table_1,0,чарт,чарт
...,...,...,...,...
3381700,dataset/Dataset/data/3660091/table_2,2,команда альфредссона,команда
3381701,dataset/Dataset/data/3660091/table_2,3,команда альфредссона,команда
3381702,dataset/Dataset/data/4212949/table_0,2,объем производства железной руды млн тонн,объём
3381703,dataset/Dataset/data/7525778/table_0,1,время выпуска,время


In [36]:
df_cleansed_lemma.drop("column_name", axis=1, inplace=True)
df_cleansed_lemma

,table_id,column_id,column_name_lemma
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/1888657/table_1,2,чарт
2,dataset/Dataset/data/6223308/table_0,0,чарт
3,dataset/Dataset/data/4293840/table_0,0,чарт
4,dataset/Dataset/data/8245389/table_1,0,чарт
...,...,...,...
3381700,dataset/Dataset/data/3660091/table_2,2,команда
3381701,dataset/Dataset/data/3660091/table_2,3,команда
3381702,dataset/Dataset/data/4212949/table_0,2,объём
3381703,dataset/Dataset/data/7525778/table_0,1,время


**Опасно, буква ё вернулась вместе с леммой :) класс.**

In [37]:
df_cleansed_lemma["column_name_lemma"] = df_cleansed_lemma["column_name_lemma"].str.replace("ё", "е")
df_cleansed_lemma

,table_id,column_id,column_name_lemma
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/1888657/table_1,2,чарт
2,dataset/Dataset/data/6223308/table_0,0,чарт
3,dataset/Dataset/data/4293840/table_0,0,чарт
4,dataset/Dataset/data/8245389/table_1,0,чарт
...,...,...,...
3381700,dataset/Dataset/data/3660091/table_2,2,команда
3381701,dataset/Dataset/data/3660091/table_2,3,команда
3381702,dataset/Dataset/data/4212949/table_0,2,объем
3381703,dataset/Dataset/data/7525778/table_0,1,время


In [38]:
empty_mask = df_cleansed_lemma["column_name_lemma"].isna()
df_cleansed_lemma[empty_mask]

,table_id,column_id,column_name_lemma
1004084,dataset/Dataset/data/8752971/table_0,1,None
1004085,dataset/Dataset/data/7323192/table_0,1,None
1004086,dataset/Dataset/data/7437687/table_0,1,None
1004087,dataset/Dataset/data/7367667/table_0,1,None
1004088,dataset/Dataset/data/8351450/table_0,1,None
...,...,...,...
3381650,dataset/Dataset/data/2623102/table_0,1,None
3381652,dataset/Dataset/data/4997442/table_0,3,None
3381653,dataset/Dataset/data/4997442/table_0,4,None
3381687,dataset/Dataset/data/5273033/table_3,4,None


In [39]:
df_cleansed_lemma = df_cleansed_lemma[~empty_mask]
df_cleansed_lemma

,table_id,column_id,column_name_lemma
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/1888657/table_1,2,чарт
2,dataset/Dataset/data/6223308/table_0,0,чарт
3,dataset/Dataset/data/4293840/table_0,0,чарт
4,dataset/Dataset/data/8245389/table_1,0,чарт
...,...,...,...
3381700,dataset/Dataset/data/3660091/table_2,2,команда
3381701,dataset/Dataset/data/3660091/table_2,3,команда
3381702,dataset/Dataset/data/4212949/table_0,2,объем
3381703,dataset/Dataset/data/7525778/table_0,1,время


In [40]:
from pathlib import Path
from tqdm import tqdm

df_cleansed_lemma.to_csv(f"./lemmatized/data.csv", sep=";", index=False)